In [10]:
from models import Customer, Account
import pandas as pd
from db import SessionLocal
session = SessionLocal()

In [11]:
# läser vi in våra normaliserade filer som vi kommer använda för att koppla till databasen
df_customers = pd.read_csv("/Users/k/Documents/TUC/DataQuality/python-bank-project-start/normalized_customers.csv")
df_accounts = pd.read_csv("/Users/k/Documents/TUC/DataQuality/python-bank-project-start/normalized_accounts.csv")

In [12]:
print("Customer columns:", Customer.__table__.columns.keys()) # dubbelkollar våra kolumner i Customer
print(df_customers.columns) # skriver ut våra kolumnnamen , vi dubbelkollar detta innan vi gör en for-loop

Customer columns: ['id', 'name', 'address', 'phone', 'personnummer']
Index(['name', 'address', 'phone', 'personnummer'], dtype='object')


In [ ]:
personnummer_to_id = {}

for _, row in df_customers.iterrows():
    # Kontrollera om kund redan finns
    existing = session.query(Customer).filter_by(personnummer=row['personnummer']).first()
    
    if existing:
        personnummer_to_id[row['personnummer']] = existing.id
        continue

    # Skapa ny kund
    customer = Customer(
        name=row['name'],
        address=row['address'],
        phone=row['phone'],
        personnummer=row['personnummer']
    )
    session.add(customer)
    session.flush()  # För att få fram autogenererat ID

    # Koppla personnummer till ID
    personnummer_to_id[row['personnummer']] = customer.id

# Commit till databasen
session.commit()
print("Kunder inlagda.")
# 


Kunder inlagda.


In [15]:
print("Account columns:", Account.__table__.columns.keys()) # dubbelkollar våra kolumner i Account
print(df_accounts.columns) # skriver ut kolumnnamnen från df_accounts, vi kontrollerar innan vi skriver vår for-looop

Account columns: ['id', 'number', 'balance', 'credit', 'interest', 'currency', 'customer_id']
Index(['number', 'personnummer', 'credit', 'interest', 'currency'], dtype='object')


In [ ]:
# här ser vi till att alla nödvändiga kolumner finns i df_accounts
required_cols = ['balance', 'credit', 'interest', 'currency']
default_values = {'balance': 0, 'credit': 0, 'interest': 0, 'currency': 'SEK'}

for col in required_cols:
    if col not in df_accounts.columns:
        df_accounts[col] = default_values[col]

In [ ]:
for _, row in df_accounts.iterrows(): 
    personnummer = row['personnummer']
    customer_id = personnummer_to_id.get(personnummer)

    if not customer_id:
        print(f"Hittade ingen kund för: {personnummer}")
        continue

    account = Account(
        number=row['number'],
        balance=row['balance'],
        credit=row['credit'],
        interest=row['interest'],
        currency=row['currency'],
        customer_id=customer_id
    )
    session.add(account)

session.commit()
print("Konton inlagda och kopplade till kunder.")



Konton inlagda och kopplade till kunder.


In [18]:
# Kollar hur många antal kunder och konton vi har 
print("Antal kunder:", session.query(Customer).count()) # vi gör en query där vi skickar en COUNT förfrågan på våran 'Customer' tabell 
print("Antal konton:", session.query(Account).count()) # samma som ovan men för 'Account' tabellen
session.close() # stänger vi vår session där inga fler databasoperationer kan göras 

Antal kunder: 1160
Antal konton: 2000
